In [ ]:
# -----------------------------------------------------------------
# STEP 1: INSTALL ALL LIBRARIES
# -----------------------------------------------------------------
print("Step 1: Installing and upgrading libraries...")
# We don't need streamlit, but we do need sentencepiece for T5
!pip install --upgrade transformers datasets evaluate rouge_score sentencepiece

Step 1: Installing and upgrading libraries...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a56bbb6ba1e8923126051d3fc03961d43b71e6f2c70e7894e22665fba8f0e267
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take i

In [ ]:
# -----------------------------------------------------------------
# STEP 2: IMPORTS
# -----------------------------------------------------------------
import datasets
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline
)
import os
import torch

In [ ]:
print("Step 3: Loading dataset and base model...")

try:
    dataset = load_dataset("cnn_dailymail", "3.0.0")
    train_dataset = dataset["train"].shuffle(seed=42).select(range(5000)) # 5k examples
    eval_dataset = dataset["validation"].shuffle(seed=42).select(range(500)) # 500 examples
    print(f"Dataset loaded. Training with {len(train_dataset)} examples.")
except Exception as e:
    print(f"Error loading dataset: {e}")

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
print("Base model 't5-small' and tokenizer loaded.")

Step 3: Loading dataset and base model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Base model 't5-small' and tokenizer loaded.


In [ ]:
print("Step 4: Setting up preprocessing function...")
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("Applying preprocessing to dataset... (This may take a minute)")
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)
print("Preprocessing complete.")


Step 4: Setting up preprocessing function...
Applying preprocessing to dataset... (This may take a minute)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Preprocessing complete.


In [ ]:
print("Step 5: Configuring the Trainer...")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
model_output_dir = "./my_summarizer_model"

# -----------------------------------------------------------------
# !! THE FIX IS HERE !!
# We are adding 'report_to="none"' to disable wandb logging.
# -----------------------------------------------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to="none",  # <--- THIS IS THE FIX
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer configured. Starting the fine-tuning process...")
try:
    trainer.train()
    print("--------------------------------------------------")
    print("🎉 TRAINING COMPLETE! 🎉")
    print("--------------------------------------------------")
except Exception as e:
    print(f"An error occurred during training: {e}")

print(f"Saving your fine-tuned model to {model_output_dir}...")
trainer.save_model(model_output_dir)
tokenizer.save_pretrained(model_output_dir)
print("Model saved successfully!")

Step 5: Configuring the Trainer...


/tmp/ipython-input-2467279046.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Trainer configured. Starting the fine-tuning process...


Epoch,Training Loss,Validation Loss
1,1.130400,1.069020
2,1.087800,1.061400
3,1.085200,1.058747


--------------------------------------------------
🎉 TRAINING COMPLETE! 🎉
--------------------------------------------------
Saving your fine-tuned model to ./my_summarizer_model...
Model saved successfully!


In [ ]:
print("\nStep 6: Testing the fine-tuned model...")

print("Loading your custom model from disk...")
model_dir = "./my_summarizer_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded onto {device}")

sample_article = """
(CNN) -- A defiant British Prime Minister Boris Johnson battled to stay in power Wednesday after the resignation of two top ministers and a slew of more junior officials, who said they could no in longer serve under his scandal-plagued leadership.
Johnson's grip on power appears to be weakening. He is facing a steady stream of resignations, beginning with his Treasury chief, Rishi Sunak, and his Health Secretary, Sajid Javid.
The pair, both heavyweights in the Cabinet, quit in quick succession on Tuesday night, prompting speculation that Johnson's premiership was nearing its end.
But a defiant Johnson vowed to "keep going" on Wednesday, despite the mass walkout and renewed calls for him to resign.
Speaking at a fiery session of Prime Minister's Questions in Parliament, Johnson said he had a "colossal mandate" from the 2019 election and would "keep going."
His press secretary told journalists that Johnson was "confident" he would still be Prime Minister on Thursday.
"""

print("\n--- SAMPLE ARTICLE ---")
print(sample_article)

prefix = "summarize: "
inputs = tokenizer(
    prefix + sample_article,
    return_tensors="pt",
    max_length=1024,
    truncation=True
).to(device)

summary_ids = model.generate(
    inputs["input_ids"],
    max_length=150,
    min_length=40,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\n--- GENERATED SUMMARY ---")
print(generated_summary)
print("\nTest complete. You can now download your model from the folder on the left.")


Step 6: Testing the fine-tuned model...
Loading your custom model from disk...
Model loaded onto cuda

--- SAMPLE ARTICLE ---

(CNN) -- A defiant British Prime Minister Boris Johnson battled to stay in power Wednesday after the resignation of two top ministers and a slew of more junior officials, who said they could no in longer serve under his scandal-plagued leadership.
Johnson's grip on power appears to be weakening. He is facing a steady stream of resignations, beginning with his Treasury chief, Rishi Sunak, and his Health Secretary, Sajid Javid.
The pair, both heavyweights in the Cabinet, quit in quick succession on Tuesday night, prompting speculation that Johnson's premiership was nearing its end.
But a defiant Johnson vowed to "keep going" on Wednesday, despite the mass walkout and renewed calls for him to resign.
Speaking at a fiery session of Prime Minister's Questions in Parliament, Johnson said he had a "colossal mandate" from the 2019 election and would "keep going."
His 

In [ ]:
print("\nStep 7: Analyzing sentiment of the article...")

# This is a great model for general sentiment, including "Neutral"
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
print(f"Loading sentiment model: {model_name}...")
print("This will download a new (RoBERTa) model...")

try:
    # We use the 'pipeline' helper to load the model
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model=model_name,
        tokenizer=model_name,
        device=0 if torch.cuda.is_available() else -1  # Use 0 for GPU, -1 for CPU
    )
    print("Sentiment model loaded successfully.")

    # Run the article through the sentiment pipeline
    # We must truncate the article, as this model has a 512-token limit
    sentiment_result = sentiment_pipeline(sample_article, truncation=True, max_length=512)

    print("\n--- GENERATED SENTIMENT ---")
    # The output is a list, let's just grab the first (and only) result
    print(f"Label: {sentiment_result[0]['label']}")
    print(f"Score: {sentiment_result[0]['score']:.4f}") # Format score to 4 decimals

except Exception as e:
    print(f"Error loading or running sentiment pipeline: {e}")
    print("This can sometimes happen if the model card is updated.")
    print("A good alternative model is 'distilbert-base-uncased-finetuned-sst-2-english'")


print("\nTest complete. You now have a summary AND a sentiment.")



Step 7: Analyzing sentiment of the article...
Loading sentiment model: cardiffnlp/twitter-roberta-base-sentiment-latest...
This will download a new (RoBERTa) model...


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Sentiment model loaded successfully.

--- GENERATED SENTIMENT ---
Label: neutral
Score: 0.5687

Test complete. You now have a summary AND a sentiment.


In [ ]:
## uploding model to hugging face

In [ ]:
print("\nStep 8: Uploading your model to the Hugging Face Hub...")

# We need to install the 'huggingface_hub' library to login
!pip install -q huggingface_hub

from huggingface_hub import notebook_login, HfApi, create_repo
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os


Step 8: Uploading your model to the Hugging Face Hub...


In [ ]:
notebook_login()


In [ ]:
# -------------------------------------------------
# 2. CONFIGURE YOUR MODEL'S NEW NAME
# -------------------------------------------------
# !!! IMPORTANT !!!
# Replace "YOUR-HF-USERNAME" with your actual Hugging Face username
# You can keep "my-summarizer-model" or change it if you like
YOUR_HF_USERNAME = "Abhiee12"
YOUR_MODEL_NAME = "my-summarizer-model"
repo_id = f"{YOUR_HF_USERNAME}/{YOUR_MODEL_NAME}"

# This is the local folder in Colab where your model is saved
local_model_dir = "./my_summarizer_model"

In [ ]:
# -------------------------------------------------
# 3. LOAD YOUR LOCAL MODEL
# -------------------------------------------------
print(f"Loading your local model from {local_model_dir}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(local_model_dir)
    model = AutoModelForSeq2SeqLM.from_pretrained(local_model_dir)
    print("Local model loaded successfully.")
except Exception as e:
    print(f"ERROR: Could not load model from {local_model_dir}.")
    print(f"Did Step 5 complete successfully and create the folder? Error: {e}")

Loading your local model from ./my_summarizer_model...
Local model loaded successfully.


In [ ]:

# -------------------------------------------------
# 4. UPLOAD THE MODEL TO THE HUB
# -------------------------------------------------
print(f"Creating/checking repository on Hugging Face: {repo_id}")

try:
    # Create the new repo on the Hub
    # (set private=True if you don't want it public)
    create_repo(repo_id, private=False, exist_ok=True)
    print(f"Repository {repo_id} created or already exists.")

    # Upload your model and tokenizer files
    print("Uploading model and tokenizer files...")
    model.push_to_hub(repo_id)
    tokenizer.push_to_hub(repo_id)

    print("\n--------------------------------------------------")
    print("🎉 YOUR MODEL IS LIVE! 🎉")
    print(f"You can see it at: https://huggingface.co/{repo_id}")
    print("--------------------------------------------------")
    print("You can now proceed with the Chrome Extension files.")
    print(f"Remember to update 'popup.js' with this repo_id: {repo_id}")

except Exception as e:
    print(f"An error occurred during the upload: {e}")

Creating/checking repository on Hugging Face: Abhiee12/my-summarizer-model
Repository Abhiee12/my-summarizer-model created or already exists.
Uploading model and tokenizer files...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0gn2j78/model.safetensors:   0%|          |  552kB /  242MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../tmp532e5izn/spiece.model: 100%|##########|  792kB /  792kB            


--------------------------------------------------
🎉 YOUR MODEL IS LIVE! 🎉
You can see it at: https://huggingface.co/Abhiee12/my-summarizer-model
--------------------------------------------------
You can now proceed with the Chrome Extension files.
Remember to update 'popup.js' with this repo_id: Abhiee12/my-summarizer-model
